In [1]:
import time
import logging
import os
from tqdm import tqdm
import numpy as np
import pandas as pd

from tinkoff.invest import Client, SecurityTradingStatus, CandleInterval
from tinkoff.invest.services import InstrumentsService
from tinkoff.invest.utils import quotation_to_decimal, now

from datetime import timedelta
import datetime

import pickle
def dump_pkl(data, filename):
  with open(filename, 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)
def load_pkl(filename):
  with open(filename, 'rb') as handle:
    data = pickle.load(handle)
  return data

from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

import requests

# 1. Global params

In [3]:
TOKEN = 'xxx'
with Client(TOKEN) as client:
    print(client.users.get_accounts())

GetAccountsResponse(accounts=[Account(id='2169433725', type=<AccountType.ACCOUNT_TYPE_TINKOFF: 1>, name='M13', status=<AccountStatus.ACCOUNT_STATUS_OPEN: 2>, opened_date=datetime.datetime(2022, 4, 27, 0, 0, tzinfo=datetime.timezone.utc), closed_date=datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), access_level=<AccessLevel.ACCOUNT_ACCESS_LEVEL_READ_ONLY: 2>)])


In [4]:
BOT_TOKEN = '7598372117:AAHKSc-jHo02zSzJ-NOoZ8GloOqdbNySeZw'
BOT_CHAT_ID = '131510115'

In [5]:
MODEL_FEATURES1 = ['close_1day_w5_norm_std/close_1day_w100_norm_std',
 'tmos_close_1day_w5_ma_up_3std/tmos_close_1day',
 'tmos_close_1day_w5_std/tmos_close_1day_w100_std',
 'close_1day_w3_std/close_1day_w100_std',
 'close_1day_w20_roc',
 'close_w70_lvl_1.02-1.03',
 'tmos_close_1day_w5_norm_std',
 'volume_1day_w20_rsi',
 'close_1day_w20_norm_std',
 'close_w14_ma_low_3std/close',
 'tmos_close_w14_max/tmos_close',
 'close_1day_w5_ma/close_1day_w100_ma',
 'tmos_close/tmos_close_w70_ma',
 'tmos_close_w70_lvl_1-1.005',
 'close_w14_mean_abs_pct/close_w70_mean_abs_pct',
 'tmos_close_1day_w20_rsi',
 'tmos_close_1day_w5_min/tmos_close_1day_w5_max',
 'close_1day_w100_lvl_1.02-1.03',
 'close_1day_w20_min/close_1day_w20_max',
 'close_w14_mean_abs_pct']


MODEL_FEATURES2 = ['tmos_close_w14_rsi',
 'tmos_close_w70_lvl_1.01-1.015',
 'tmos_close_w70_lvl_-0.985-0.99',
 'tmos_close_1day_w20_lvl_1.005-1.01',
 'tmos_close_1day_w100_lvl_1.04-1.05',
 'tmos_close_w70_rsi',
 'tmos_close_w70_roc',
 'close_w70_rsi',
 'hour',
 'tmos_close_w70_ma_up_2std/tmos_close',
 'tmos_close_w14_ma/tmos_close_w70_ma',
 'close_w70_lvl_-0.98-0.99',
 'tmos_close_w5_ma/tmos_close_w70_ma',
 'tmos_close_w14_max/tmos_close',
 'tmos_close_1day_w100_lvl_1.015-1.02',
 'tmos_close_1day_w20_lvl_1.015-1.02',
 'tmos_close_w5_max/tmos_close',
 'close_w1_roc',
 'tmos_close_1day_w3_mean_abs_pct/tmos_close_1day_w5_mean_abs_pct',
 'tmos_close_1day_w5_ma_low_2std/tmos_close_1day']

MODEL_FEATURES3 = []

FEATURES = list(set(MODEL_FEATURES1 + MODEL_FEATURES2 + MODEL_FEATURES3))

In [6]:
len(FEATURES)

39

In [7]:

COUNT_DAY_POINTS = 100
COUNT_HOUR_POINTS = 70




STOCKS = ['AFLT',
 'IRAO',
 'MVID',
 'OGKB',
 'SPBE',
 'MTLRP',
 'SFIN',
 'KMAZ',
 'CBOM',
 'MSNG',
 'FEES',
 'LKOH',
 'FESH',
 'KROT',
 'LENT',
 'MDMG',
 'NKNCP',
 'VKCO',
 'UPRO',
 'MRKY',
 'SELG',
 'SGZH',
 'AKRN',
 'CHMK',
 'ENPG',
 'YAKG',
 'PMSBP',
 'RKKE',
 'GEMC',
 'LSRG',
 'MRKZ',
 'BANE',
 'CNTL',
 'PMSB',
 'KRKNP',
 'NMTP',
 'PLZL',
 'TATN',
 'AFKS',
 'UNKL',
 'APTK',
 'LNZL',
 'GMKN',
 'NSVZ',
 'LIFE',
 'RNFT',
 'VRSB',
 'YDEX',
 'UWGN',
 'TGKN',
 'ABRD',
 'HYDR',
 'ABIO',
 'WUSH',
 'RTKMP',
 'GTRK',
 'NLMK',
 'TATNP',
 'CNTLP',
 'BLNG',
 'ALRS',
 'GCHE',
 'VSMO',
 'LSNG',
 'TRNFP',
 'MGNT',
 'RENI',
 'NKHP',
 'TGKB',
 'ROLO',
 'IRKT',
 'MGTSP',
 'MTLR',
 'TGKA',
 'MSRS',
 'HEAD',
 'RUAL',
 'MRKV',
 'KLSB',
 'SIBN',
 'SNGS',
 'MSTT',
 'KAZTP',
 'DVEC',
 'NVTK',
 'RASP',
 'VTBR',
 'PHOR',
 'TTLK',
 'NKNC',
 'T',
 'KAZT',
 'AMEZ',
 'MRKU',
 'TRMK',
 'MOEX',
 'SBERP',
 'X5',
 'CHMF',
 'RTKM',
 'SMLT',
 'LNZLP',
 'MRKC',
 'MRKP',
 'UNAC',
 'KZOSP',
 'GAZP',
 'ROSN',
 'BELU',
 'ELFV',
 'FLOT',
 'PIKK',
 'LSNGP',
 'SVAV',
 'TGKBP',
 'POSI',
 'KZOS',
 'MRKS',
 'SNGSP',
 'MTSS',
 'MAGN',
 'PRFN',
 'SBER',
 'BANEP',
 'BSPB',
 'AQUA',
 'RBCM']

In [8]:
len(STOCKS)

127

In [9]:
GROUPS_1HOUR = [
        #1hour: w1, #w5, #w14, #w70
         ['close_w1_roc', 'close_w5_alpha', 'close_w14_alpha', 'close_w70_alpha'],
         ['volume_w1_roc', 'volume_w5_alpha', 'volume_w14_alpha', 'volume_w70_alpha'],
         ['tmos_close_w1_roc', 'tmos_close_w5_alpha', 'tmos_close_w14_alpha', 'tmos_close_w70_alpha'],

         ['close_w1_roc', 'close_w5_roc', 'close_w14_roc', 'close_w70_roc'],
         ['volume_w1_roc', 'volume_w5_roc', 'volume_w14_roc', 'volume_w70_roc'],
         ['tmos_close_w1_roc', 'tmos_close_w5_roc', 'tmos_close_w14_roc', 'tmos_close_w70_roc'],

         ['close_w5_mean_abs_pct', 'close_w14_mean_abs_pct', 'close_w70_mean_abs_pct'],
         ['volume_w5_mean_abs_pct', 'volume_w14_mean_abs_pct', 'volume_w70_mean_abs_pct'],
         ['tmos_close_w5_mean_abs_pct', 'tmos_close_w14_mean_abs_pct', 'tmos_close_w70_mean_abs_pct'],

         ['close_w5_std', 'close_w14_std', 'close_w70_std'],
         ['volume_w5_std', 'volume_w14_std', 'volume_w70_std'],
         ['tmos_close_w5_std', 'tmos_close_w14_std', 'tmos_close_w70_std'],

         ['close_w5_norm_std', 'close_w14_norm_std', 'close_w70_norm_std'],
         ['volume_w5_norm_std', 'volume_w14_norm_std', 'volume_w70_norm_std'],
         ['tmos_close_w5_norm_std', 'tmos_close_w14_norm_std', 'tmos_close_w70_norm_std'],

         ['close_w5_rsi', 'close_w14_rsi', 'close_w70_rsi'],
         ['volume_w5_rsi', 'volume_w14_rsi', 'volume_w70_rsi'],
         ['tmos_close_w5_rsi', 'tmos_close_w14_rsi', 'tmos_close_w70_rsi'],
        
         ['close', 'close_w5_ma', 'close_w14_ma', 'close_w70_ma'],
         ['volume', 'volume_w5_ma', 'volume_w14_ma', 'volume_w70_ma'],
         ['tmos_close', 'tmos_close_w5_ma', 'tmos_close_w14_ma', 'tmos_close_w70_ma'],

         {'close' : ['close_w5_min', 'close_w14_min', 'close_w70_min']},
         {'volume' : ['volume_w5_min', 'volume_w14_min', 'volume_w70_min']},
         {'tmos_close' : ['tmos_close_w5_min', 'tmos_close_w14_min', 'tmos_close_w70_min']},
    
         {'close' : ['close_w5_max', 'close_w14_max', 'close_w70_max']},
         {'volume' : ['volume_w5_max', 'volume_w14_max', 'volume_w70_max']},
         {'tmos_close' : ['tmos_close_w5_max', 'tmos_close_w14_max', 'tmos_close_w70_max']},


        #w5
         {'close' : [  'close_w5_ma_low_2std', 'close_w5_ma_up_2std', 'close_w5_ma_low_3std', 'close_w5_ma_up_3std']},
         {'volume' : [  'volume_w5_ma_low_2std', 'volume_w5_ma_up_2std', 'volume_w5_ma_low_3std', 'volume_w5_ma_up_3std']},
         {'tmos_close' : [  'tmos_close_w5_ma_low_2std', 'tmos_close_w5_ma_up_2std', 'tmos_close_w5_ma_low_3std', 'tmos_close_w5_ma_up_3std']},
        #w14
         {'close' : [ 'close_w14_ma_low_2std', 'close_w14_ma_up_2std', 'close_w14_ma_low_3std', 'close_w14_ma_up_3std']},
         {'volume' : [ 'volume_w14_ma_low_2std', 'volume_w14_ma_up_2std', 'volume_w14_ma_low_3std', 'volume_w14_ma_up_3std']},
         {'tmos_close' : [ 'tmos_close_w14_ma_low_2std', 'tmos_close_w14_ma_up_2std', 'tmos_close_w14_ma_low_3std', 'tmos_close_w14_ma_up_3std']},
        #w70
         {'close' : [ 'close_w70_ma_low_2std', 'close_w70_ma_up_2std', 'close_w70_ma_low_3std', 'close_w70_ma_up_3std']},
         {'volume' : [ 'volume_w70_ma_low_2std', 'volume_w70_ma_up_2std', 'volume_w70_ma_low_3std', 'volume_w70_ma_up_3std']},
         {'tmos_close' : [ 'tmos_close_w70_ma_low_2std', 'tmos_close_w70_ma_up_2std', 'tmos_close_w70_ma_low_3std', 'tmos_close_w70_ma_up_3std']},


    #comment
        #w5
        ['close_w5_min', 'close_w5_max'],
        ['volume_w5_min', 'volume_w5_max'],
        ['tmos_close_w5_min', 'tmos_close_w5_max'],
        #w14
        ['close_w14_min', 'close_w14_max'],
        ['volume_w14_min', 'volume_w14_max'],
        ['tmos_close_w14_min', 'tmos_close_w14_max'],
        #w70
        ['close_w70_min', 'close_w70_max'],
        ['volume_w70_min', 'volume_w70_max'],
        ['tmos_close_w70_min', 'tmos_close_w70_max'],
]

GROUPS_1DAY = [
        #1day: #w1, #w3, #w5, #w20, #w100
         ['close_1day_w1_roc', 'close_1day_w3_alpha', 'close_1day_w5_alpha', 'close_1day_w20_alpha', 'close_1day_w100_alpha'],
         ['volume_1day_w1_roc','volume_1day_w3_alpha', 'volume_1day_w5_alpha', 'volume_1day_w20_alpha', 'volume_1day_w100_alpha'],
         ['tmos_close_1day_w1_roc', 'tmos_close_1day_w3_alpha', 'tmos_close_1day_w5_alpha', 'tmos_close_1day_w20_alpha', 'tmos_close_1day_w100_alpha'],

         ['close_1day_w1_roc', 'close_1day_w3_roc', 'close_1day_w5_roc', 'close_1day_w20_roc', 'close_1day_w100_roc'],
         ['volume_1day_w1_roc', 'volume_1day_w3_roc', 'volume_1day_w5_roc', 'volume_1day_w20_roc', 'volume_1day_w100_roc'],
         ['tmos_close_1day_w1_roc','tmos_close_1day_w3_roc', 'tmos_close_1day_w5_roc', 'tmos_close_1day_w20_roc', 'tmos_close_1day_w100_roc'],

         ['close_1day_w3_mean_abs_pct', 'close_1day_w5_mean_abs_pct', 'close_1day_w20_mean_abs_pct', 'close_1day_w100_mean_abs_pct'],
         ['volume_1day_w3_mean_abs_pct','volume_1day_w5_mean_abs_pct', 'volume_1day_w20_mean_abs_pct', 'volume_1day_w100_mean_abs_pct'],
         ['tmos_close_1day_w3_mean_abs_pct', 'tmos_close_1day_w5_mean_abs_pct', 'tmos_close_1day_w20_mean_abs_pct', 'tmos_close_1day_w100_mean_abs_pct'],

         ['close_1day_w3_std', 'close_1day_w5_std', 'close_1day_w20_std', 'close_1day_w100_std'],
         ['volume_1day_w3_std', 'volume_1day_w5_std', 'volume_1day_w20_std', 'volume_1day_w100_std'],
         ['tmos_close_1day_w3_std', 'tmos_close_1day_w5_std', 'tmos_close_1day_w20_std', 'tmos_close_1day_w100_std'],

         ['close_1day_w3_norm_std', 'close_1day_w5_norm_std', 'close_1day_w20_norm_std', 'close_1day_w100_norm_std'],
         ['volume_1day_w3_norm_std', 'volume_1day_w5_norm_std', 'volume_1day_w20_norm_std', 'volume_1day_w100_norm_std'],
         ['tmos_close_1day_w3_norm_std', 'tmos_close_1day_w5_norm_std', 'tmos_close_1day_w20_norm_std', 'tmos_close_1day_w100_norm_std'],

         ['close_1day_w3_rsi',  'close_1day_w5_rsi', 'close_1day_w20_rsi', 'close_1day_w100_rsi'],
         ['volume_1day_w3_rsi', 'volume_1day_w5_rsi', 'volume_1day_w20_rsi', 'volume_1day_w100_rsi'],
         ['tmos_close_1day_w3_rsi', 'tmos_close_1day_w5_rsi', 'tmos_close_1day_w20_rsi', 'tmos_close_1day_w100_rsi'],
        
         ['close_1day','close_1day_w3_ma', 'close_1day_w5_ma', 'close_1day_w20_ma', 'close_1day_w100_ma'],
         ['volume_1day', 'volume_1day_w3_ma', 'volume_1day_w5_ma', 'volume_1day_w20_ma', 'volume_1day_w100_ma'],
         ['tmos_close_1day','tmos_close_1day_w3_ma', 'tmos_close_1day_w5_ma', 'tmos_close_1day_w20_ma', 'tmos_close_1day_w100_ma'],

         {'close_1day' : ['close_1day_w3_min', 'close_1day_w5_min', 'close_1day_w20_min', 'close_1day_w100_min']},
         {'volume_1day' : ['volume_1day_w3_min', 'volume_1day_w5_min', 'volume_1day_w20_min', 'volume_1day_w100_min']},
         {'tmos_close_1day' : ['tmos_close_1day_w3_min', 'tmos_close_1day_w5_min', 'tmos_close_1day_w20_min', 'tmos_close_1day_w100_min']},
    
         {'close_1day' : ['close_1day_w3_max', 'close_1day_w5_max', 'close_1day_w20_max', 'close_1day_w100_max']},
         {'volume_1day' : ['volume_1day_w3_max', 'volume_1day_w5_max', 'volume_1day_w20_max', 'volume_1day_w100_max']},
         {'tmos_close_1day' : ['tmos_close_1day_w3_max','tmos_close_1day_w5_max', 'tmos_close_1day_w20_max', 'tmos_close_1day_w100_max']},

        #w3
         {'close_1day' : [  'close_1day_w3_ma_low_2std', 'close_1day_w3_ma_up_2std', 'close_1day_w3_ma_low_3std', 'close_1day_w3_ma_up_3std']},
         {'volume_1day' : [  'volume_1day_w3_ma_low_2std', 'volume_1day_w3_ma_up_2std', 'volume_1day_w3_ma_low_3std', 'volume_1day_w3_ma_up_3std']},
         {'tmos_close_1day' : [  'tmos_close_1day_w3_ma_low_2std', 'tmos_close_1day_w3_ma_up_2std', 'tmos_close_1day_w3_ma_low_3std', 'tmos_close_1day_w3_ma_up_3std']},
        #w5
         {'close_1day' : [  'close_1day_w5_ma_low_2std', 'close_1day_w5_ma_up_2std', 'close_1day_w5_ma_low_3std', 'close_1day_w5_ma_up_3std']},
         {'volume_1day' : [  'volume_1day_w5_ma_low_2std', 'volume_1day_w5_ma_up_2std', 'volume_1day_w5_ma_low_3std', 'volume_1day_w5_ma_up_3std']},
         {'tmos_close_1day' : [ 'tmos_close_1day_w5_ma_low_2std', 'tmos_close_1day_w5_ma_up_2std', 'tmos_close_1day_w5_ma_low_3std', 'tmos_close_1day_w5_ma_up_3std']},
        #w20
         {'close_1day' : [ 'close_1day_w20_ma_low_2std', 'close_1day_w20_ma_up_2std', 'close_1day_w20_ma_low_3std', 'close_1day_w20_ma_up_3std']},
         {'volume_1day' : [ 'volume_1day_w20_ma_low_2std', 'volume_1day_w20_ma_up_2std', 'volume_1day_w20_ma_low_3std', 'volume_1day_w20_ma_up_3std']},
         {'tmos_close_1day' : [ 'tmos_close_1day_w20_ma_low_2std', 'tmos_close_1day_w20_ma_up_2std', 'tmos_close_1day_w20_ma_low_3std', 'tmos_close_1day_w20_ma_up_3std']},
        #w100
         {'close_1day' : [ 'close_1day_w100_ma_low_2std', 'close_1day_w100_ma_up_2std', 'close_1day_w100_ma_low_3std', 'close_1day_w100_ma_up_3std']},
         {'volume_1day' : [ 'volume_1day_w100_ma_low_2std', 'volume_1day_w100_ma_up_2std', 'volume_1day_w100_ma_low_3std', 'volume_1day_w100_ma_up_3std']},
         {'tmos_close_1day' : ['tmos_close_1day_w100_ma_low_2std', 'tmos_close_1day_w100_ma_up_2std', 'tmos_close_1day_w100_ma_low_3std', 'tmos_close_1day_w100_ma_up_3std']},

    #comment
        #w3
        ['close_1day_w3_min', 'close_1day_w3_max'],
        ['volume_1day_w3_min', 'volume_1day_w3_max'],
        ['tmos_close_1day_w3_min', 'tmos_close_1day_w3_max'],    
        #w5
        ['close_1day_w5_min', 'close_1day_w5_max'],
        ['volume_1day_w5_min', 'volume_1day_w5_max'],
        ['tmos_close_1day_w5_min', 'tmos_close_1day_w5_max'],
    
        #w20
        ['close_1day_w20_min', 'close_1day_w20_max'],
        ['volume_1day_w20_min', 'volume_1day_w20_max'],
        ['tmos_close_1day_w20_min', 'tmos_close_1day_w20_max'],
    
        # #w100 - вот эти фичи бы удалить - это скорее лик
        # ['close_1day_w100_min', 'close_1day_w100_max'],
        # ['volume_1day_w100_min', 'volume_1day_w100_max'],
        # ['tmos_close_1day_w100_min', 'tmos_close_1day_w100_max'],
]

# 2. Make dirs

In [11]:
!mkdir data

mkdir: data: File exists


In [12]:
!mkdir data/raw data/preproc data/lgbm data/result

mkdir: data/raw: File exists
mkdir: data/preproc: File exists
mkdir: data/lgbm: File exists
mkdir: data/result: File exists


In [13]:
!mkdir data/raw/1day data/raw/1hour data/preproc/1day data/preproc/1hour

mkdir: data/raw/1day: File exists
mkdir: data/raw/1hour: File exists
mkdir: data/preproc/1day: File exists
mkdir: data/preproc/1hour: File exists


# 2. Get data

In [15]:
def write_logs(text):
    with open("logs.txt", "a") as f:
        f.write(text)

In [16]:
def load_figi():
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'Start loading figi data' +'\n')
    #make dir

    count_tries = 0
    max_count_tries = 5
    flag_data = False
    while (not flag_data) and (count_tries < max_count_tries):
        count_tries += 1
        try:
            with Client(TOKEN) as client:
                instruments: InstrumentsService = client.instruments
                tickers = []
                for method in ["shares", "bonds", "etfs", "currencies", "futures"]:
                    for item in getattr(instruments, method)().instruments:
                        tickers.append(
                            {
                                "name": item.name,
                                "ticker": item.ticker,
                                "class_code": item.class_code,
                                "figi": item.figi,
                                "uid": item.uid,
                                "type": method,
                                "min_price_increment": quotation_to_decimal(
                                    item.min_price_increment
                                ),
                                "scale": 9 - len(str(item.min_price_increment.nano)) + 1,
                                "lot": item.lot,
                                "trading_status": str(
                                    SecurityTradingStatus(item.trading_status).name
                                ),
                                "api_trade_available_flag": item.api_trade_available_flag,
                                "currency": item.currency,
                                "exchange": item.exchange,
                                "buy_available_flag": item.buy_available_flag,
                                "sell_available_flag": item.sell_available_flag,
                                "short_enabled_flag": item.short_enabled_flag,
                                "klong": quotation_to_decimal(item.klong),
                                "kshort": quotation_to_decimal(item.kshort),
                            }
                        )
        
            tickers_df = pd.DataFrame(tickers)
            dump_pkl(tickers_df, './data/data_figi.pkl')
            
            flag_data = True
        except:
            time.sleep(5)

    if not flag_data:
        write_logs(str(pd.Timestamp.now().round('s')) + ' '+ f'Failed to load figi data' +'\n')


    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'End loading figi data' +'\n')

def get_figi(ticker):

        df_figi = load_pkl('./data/data_figi.pkl')

        df_figi = df_figi[df_figi["ticker"] == ticker]
        if ticker == 'T': # таких два тикера T
            df_figi = df_figi[df_figi['currency'] == 'rub']
            
        assert df_figi.shape[0] == 1, f"{ticker}: error figi"

        figi = df_figi["figi"].iloc[0]
        return figi

In [17]:
def get_all_candles(ticker=None, from_=None, to_=None, interval=None):
    
    figi = get_figi(ticker)
    
    with Client(TOKEN) as client:
        
        data = [elem for elem in client.get_all_candles(figi=figi, 
                                      from_=from_-timedelta(hours=3),
                                      to=to_- timedelta(hours=3),
                                     interval=interval
                                    )
               ]

    
    open_ = np.zeros(len(data))
    high = np.zeros(len(data))
    low = np.zeros(len(data))
    close = np.zeros(len(data))
    volume = np.zeros(len(data))
    time = np.zeros(len(data), dtype='O')
    for i, elem in enumerate(data):
        open_[i] = quotation_to_decimal(elem.open)
        high[i] = quotation_to_decimal(elem.high)
        low[i] = quotation_to_decimal(elem.low)
        close[i] = quotation_to_decimal(elem.close)
        volume[i] = elem.volume
        time[i] = elem.time


    df = pd.DataFrame({'time' : time,
                      'open' : open_,
                      'close' : close,
                      'volume' : volume,
                      'low' : low,
                      'high' : high
                    })
    df['time'] += pd.Timedelta(hours=3)
    return df

In [18]:
# %%time

#load_figi()

# ticker = 'SFIN'
# #2024-06-10 
# data = get_all_candles(ticker=ticker,
#                        from_=datetime.datetime(2024, 12, 29, tzinfo=datetime.timezone.utc),
#                        to_=datetime.datetime(2025, 1, 29, tzinfo=datetime.timezone.utc),
#                        interval=CandleInterval.CANDLE_INTERVAL_HOUR
#                       )

# data['ticker'] = ticker
# data

In [19]:
# %%time

#load_figi()


# ticker = 'SFIN'
# #2024-06-10 
# data = get_all_candles(ticker=ticker,
#                        from_=datetime.datetime(2025, 1, 20, tzinfo=datetime.timezone.utc),
#                        to_=datetime.datetime(2025, 1, 29, tzinfo=datetime.timezone.utc),
#                        interval=CandleInterval.CANDLE_INTERVAL_DAY
#                       )

# data['ticker'] = ticker
# data

In [20]:
def is_business_day(date):
    return bool(len(pd.bdate_range(date, date)))


def get_data_1day(stocks):
    #loading figi
    load_figi()

    
    #logs
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'Start loading daily data' +'\n')    
    
    #make dir
    time_name = str(pd.Timestamp.now().date()) 
    if not os.path.exists(f'./data/raw/1day/{time_name}'):
        os.mkdir(f'./data/raw/1day/{time_name}')

    #loading data
    end = pd.Timestamp.now()+pd.Timedelta(days=1) #+1day, чтобы выгрузить все, что есть (+1 точка для проверки актуальности данных)
    start = end - pd.Timedelta(days=200)
    for ticker in tqdm(stocks+['TMOS']): 
        
        count_tries = 0
        max_count_tries = 3
        flag_data = False
        while (not flag_data) and (count_tries < max_count_tries):
            count_tries += 1
            try:
                data = get_all_candles(ticker=ticker,
                               from_=start,
                               to_=end,
                               interval=CandleInterval.CANDLE_INTERVAL_DAY)
                flag_data = True
            except:
                time.sleep(3)    
        #saving
        if flag_data:
            data = data[['time', 'close', 'volume']]
            data['ticker'] = ticker
            dump_pkl(data, f'./data/raw/1day/{time_name}/{ticker}.pkl')
        else:
            write_logs(str(pd.Timestamp.now().round('s')) + ' '+ f'{ticker}: failed to load daily data' +'\n')
            
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'End loading daily data' +'\n')



def get_data_1hour(stocks):
    #logs
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'Start loading hourly data' +'\n')

    #make dir
    time_name = str(pd.Timestamp.now()).split(':')[0].replace(' ', '_') #date+hour
    if not os.path.exists(f'./data/raw/1hour/{time_name}'):
        os.mkdir(f'./data/raw/1hour/{time_name}')
    
    #loading data
    end = pd.Timestamp.now()+pd.Timedelta(days=1) #+1day, чтобы выгрузить все, что есть (+1 точка для проверки целостности данных)
    start = end - pd.Timedelta(days=15)
    for ticker in tqdm(stocks+['TMOS']): 
        
        count_tries = 0
        max_count_tries = 3
        flag_data = False
        while (not flag_data) and (count_tries < max_count_tries):
            count_tries += 1
            try:
                data = get_all_candles(ticker=ticker,
                               from_=start,
                               to_=end,
                               interval=CandleInterval.CANDLE_INTERVAL_HOUR)
                flag_data = True
            except:
                time.sleep(3)    
        #saving
        if flag_data:
            data = data[['time', 'close', 'volume']]
            data['ticker'] = ticker
            dump_pkl(data, f'./data/raw/1hour/{time_name}/{ticker}.pkl')
        else:
            write_logs(str(pd.Timestamp.now().round('s')) + ' '+ f'{ticker}: failed to load hourly data' +'\n')

    
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'End loading hourly data' +'\n')


    

In [21]:
#get_data_1day(STOCKS)

In [22]:
#get_data_1hour(STOCKS)

In [23]:
# df = load_pkl('./data/raw/1hour/2025-01-29_14/NLMK.pkl')
# df

# 4. Preproc data

In [25]:
def is_business_day(date):
    return bool(len(pd.bdate_range(date, date)))

def time_cut_data_1day(data):
    data['time'] = data['time'].apply(lambda x: x.tz_convert(None))
    
    mask_bd = np.array(data['time'].apply(lambda x: is_business_day(x)))
    data = data.loc[mask_bd, :]
    data.reset_index(drop=True, inplace=True)

    return data

def preproc_data_1day(stocks):
    #logs
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'Start preproc daily data' +'\n')

    #make dir
    time_name = str(pd.Timestamp.now().date()) 
    if not os.path.exists(f'./data/preproc/1day/{time_name}'):
        os.mkdir(f'./data/preproc/1day/{time_name}')

    #tmos
    if not os.path.exists(f"./data/raw/1day/{time_name}/TMOS.pkl"):
        write_logs(str(pd.Timestamp.now().round('s')) + ' '+ f'TMOS: failed to preproc daily data' +'\n')
        return
    data_tmos = load_pkl(f"./data/raw/1day/{time_name}/TMOS.pkl")

    #preproc
    for ticker in tqdm(stocks):
        try:
            data = load_pkl(f"./data/raw/1day/{time_name}/{ticker}.pkl")
        
            data_ext = data.merge(data_tmos[['time', 'close']].copy().rename(columns={"close" : "tmos_close"}), how='left', on='time')
            assert data.shape[0] == data_ext.shape[0], 'Error join tmos'
            
            data = time_cut_data_1day(data_ext)
            assert data['tmos_close'].isnull().sum() == 0, 'Error tmos_close nulls' #праздники, в них индекс не торгуется (мб в послед время рассчитывется?)
            data['tmos_close'] = data['tmos_close'].ffill()

            #Actual time date (не самая идеальная реализация, так как чекает наличие следующую точку после прогнозной, а не прогнозную)
            last_time = data['time'].iloc[-1]
            assert last_time.date() == pd.Timestamp.now().date(), 'Error time_data' 
            
            #Обрежем последнюю точку 
            mask = data['time'] < pd.Timestamp.now().floor('d')
            data = data[mask]
            
            data = data.iloc[-(COUNT_DAY_POINTS+1):]
            data.reset_index(drop=True, inplace=True)

            #len
            assert data.shape[0] == (COUNT_DAY_POINTS+1), 'Error len'
            
            #Заполняемость
            diff_time_day = data['time'].diff() / pd.Timedelta(hours=24)
            assert (diff_time_day.iloc[1:] % 1 == 0).all(), 'Error filling 1'
            assert (diff_time_day.iloc[1:] == 2).sum() < 5, 'Error filling 2'
            # assert (diff_time_day.iloc[1:] < 10).all(), 'Error 5' #переезд акций ломает тест

            #duplicates in time
            assert (diff_time_day != 0).all(), 'Error time duplicates'

            #sort in time
            assert (diff_time_day.iloc[1:] > 0).all(), 'Error time sort'

            #Nulls
            assert not data.isnull().any().any(), 'Error nulls'  
            
            dump_pkl(data, f'./data/preproc/1day/{time_name}/{ticker}.pkl')
        except Exception as exception:
            write_logs(str(pd.Timestamp.now().round('s')) + ' '+ f'{ticker}: failed to preproc daily data ({exception})' +'\n')

    #logs
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'End preproc daily data' +'\n')






def time_cut_data_1hour(data):
    data['time'] = data['time'].apply(lambda x: x.tz_convert(None))
    
    mask_bd = np.array(data['time'].apply(lambda x: is_business_day(x)))
    data = data.loc[mask_bd, :]

    mask_volume = np.array(datetime.time(10, 0) <= pd.to_datetime(data['time'], format='%H:%M').dt.time)
    data = data.loc[mask_volume, :]
    data.reset_index(drop=True, inplace=True)
    
    return data

def preproc_data_1hour(stocks):
    #logs
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'Start preproc hourly data' +'\n')

    #make dir
    time_name = str(pd.Timestamp.now()).split(':')[0].replace(' ', '_') #date+hour
    if not os.path.exists(f'./data/preproc/1hour/{time_name}'):
        os.mkdir(f'./data/preproc/1hour/{time_name}')

    #tmos
    if not os.path.exists(f"./data/raw/1hour/{time_name}/TMOS.pkl"):
        write_logs(str(pd.Timestamp.now().round('s')) + ' '+ f'TMOS: failed to preproc hourly data' +'\n')
        return
    data_tmos = load_pkl(f"./data/raw/1hour/{time_name}/TMOS.pkl")
    #костыль
    if data_tmos['time'].iloc[-1] == data_tmos['time'].iloc[-2]:
        data_tmos = data_tmos.iloc[:-1].reset_index(drop=True)

    #preproc
    for ticker in tqdm(stocks):
        try:
            data = load_pkl(f"./data/raw/1hour/{time_name}/{ticker}.pkl")
        
            data_ext = data.merge(data_tmos[['time', 'close']].copy().rename(columns={"close" : "tmos_close"}), how='left', on='time')
            assert data.shape[0] == data_ext.shape[0], 'Error join tmos'
        
            data = time_cut_data_1hour(data_ext)
            assert data['tmos_close'].isnull().sum() < 50, 'Error tmos_close nulls' #праздники, в них индекс не торгуется
            data['tmos_close'] = data['tmos_close'].ffill()

            #Actual time date (не самая идеальная реализация, так как чекает наличие следующую точку после прогнозной, а не прогнозную)
            last_time = data['time'].iloc[-1]
            assert (last_time == pd.Timestamp.now().floor('h')) or\
                            (last_time == (pd.Timestamp.now() - pd.Timedelta(hours=1)).floor('h')), 'Error time_data' #послабление для низколиквидных акций
            
            #Обрежем последнюю точку (или две, они бывают задваиваются - особенность онлайн данных)
            mask = data['time'] < pd.Timestamp.now().floor('h')
            data = data[mask]
            
            data = data.iloc[-(COUNT_HOUR_POINTS+1):]
            data.reset_index(drop=True, inplace=True)

            #len
            assert data.shape[0] == (COUNT_HOUR_POINTS+1), 'Error len'
            
            #Заполняемость
            diff_time_hour = data['time'].diff() / pd.Timedelta(hours=1)
            assert (diff_time_hour.iloc[1:] % 1 == 0).all(), 'Error filling 1'
            assert (diff_time_hour.iloc[1:] == 2).sum() < 10, 'Error filling 2'
            assert (diff_time_hour.iloc[1:] < 24*5).all(), 'Error filling 3' #переезд акций

            #duplicates in time
            assert (diff_time_hour != 0).all(), 'Error time duplicates'

            #sort in time
            assert (diff_time_hour.iloc[1:] > 0).all(), 'Error time sort'
            
            #Nulls
            assert not data.isnull().any().any(), 'Error nulls'
    
        
            dump_pkl(data, f'./data/preproc/1hour/{time_name}/{ticker}.pkl')
        except Exception as exception:
            write_logs(str(pd.Timestamp.now().round('s')) + ' '+ f'{ticker}: failed to preproc hourly data ({exception})' +'\n')

    #logs
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'End preproc hourly data' +'\n')



In [26]:
# preproc_data_1day(STOCKS)

In [27]:
# preproc_data_1hour(STOCKS)

In [28]:
# data = load_pkl('./data/preproc/1day/2025-01-29/YDEX.pkl')
# data

In [29]:
# data = load_pkl('./data/raw/1hour/2025-02-03_10/AKRN.pkl')
# data

In [30]:
#get_data_1hour(STOCKS)

# 5. Prepare data to lgbm

In [32]:
from sklearn.linear_model import LinearRegression

def calculate_bollinger_bands(data, window):
    """Calculate Bollinger Bands"""
    rolling_mean = data.rolling(window=window, min_periods=1).mean().values
    rolling_std = data.rolling(window=window, min_periods=1).std().values

    norm_rolling_std = rolling_std/rolling_mean

    num_of_std = 2
    lower_band_2std = rolling_mean - (rolling_std * num_of_std)
    upper_band_2std = rolling_mean + (rolling_std * num_of_std)
    
    num_of_std = 3
    lower_band_3std = rolling_mean - (rolling_std * num_of_std)
    upper_band_3std = rolling_mean + (rolling_std * num_of_std)
    
    
    return rolling_mean, rolling_std, norm_rolling_std, lower_band_2std, upper_band_2std, lower_band_3std, upper_band_3std

def calculate_rsi(data, window):
    """Calculate Relative Strength Index"""
    delta = data.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    mask = avg_loss == 0
    rsi[mask] = 100
    
    return rsi.values

def calculate_roc(data, periods):
    """Calculate Rate of Change."""
    roc = ((data - data.shift(periods)) / data.shift(periods))
    return roc.values



def calc_stats(data, window=None, feat_name=None):
    #mean, std
    rolling_mean, rolling_std, norm_rolling_std,\
    lower_band_2std, upper_band_2std, lower_band_3std, upper_band_3std = calculate_bollinger_bands(data, window)

    #mean_abs_pct
    mean_abs_pct = data.pct_change(periods=1).rolling(window=window, min_periods=1).apply(lambda x: x.abs().mean())
        
    #alpha
    alpha = data.rolling(window=window, min_periods=2).apply(lambda x: LinearRegression().fit(x.values.reshape(-1, 1), np.arange(x.shape[0])).coef_[0])

    #min, max
    rolling_min = data.rolling(window=window, min_periods=1).min().values
    rolling_max = data.rolling(window=window, min_periods=1).max().values
    
    #rsi
    rsi = calculate_rsi(data, window)
    
    #roc
    roc = calculate_roc(data, window)
    diff = data.diff(window).values

    #можно угол угла наклона добавить, чтобы определять фазы рынка
    
    df_features = pd.DataFrame({f'{feat_name}_ma' : rolling_mean,
                        f'{feat_name}_std' : rolling_std,
                        f'{feat_name}_norm_std' : norm_rolling_std,
                        f'{feat_name}_ma_low_2std' : lower_band_2std,
                        f'{feat_name}_ma_up_2std' : upper_band_2std,
                        f'{feat_name}_ma_low_3std' : lower_band_3std,
                        f'{feat_name}_ma_up_3std' : upper_band_3std, 

                        f'{feat_name}_mean_abs_pct' : mean_abs_pct,
                            
                        f'{feat_name}_alpha' : alpha,
                            
                        f'{feat_name}_min' : rolling_min,
                        f'{feat_name}_max' : rolling_max,
                        f'{feat_name}_rsi' : rsi,
                        f'{feat_name}_roc' : roc,
                        f'{feat_name}_diff' : diff,
                        })
    return df_features


def calc_stats_diff_1(data, feat_name=None):
    return pd.DataFrame({f'{feat_name}_roc' : data.pct_change(periods=1).values,
                        f'{feat_name}_diff' : data.diff(1).values,
                        })

def calc_levels(data, window=None, levels=None, feat_name=None):
    
    #уровни
    data_levels = []
    column_names = []
    for i in range(1, len(levels)):
        level_low = levels[i-1]
        level_high = levels[i]
        data_levels += [data.rolling(window=window, min_periods=1).apply(lambda x: (((1+level_low)*x.values[-1] < x.values) & (x.values <= (1+level_high)*x.values[-1])).sum()).values]
        data_levels += [data.rolling(window=window, min_periods=1).apply(lambda x: (((1-level_high)*x.values[-1] <= x.values) & (x.values < (1-level_low)*x.values[-1])).sum()).values]

        column_names += [f"{feat_name}_lvl_{1+level_low}-{1+level_high}"]
        column_names += [f"{feat_name}_lvl_-{1-level_high}-{1-level_low}"]
    df_levels = pd.DataFrame({column_names[i]:data_levels[i] for i in range(len(column_names))})
    return df_levels


In [33]:
def calculate_features_1day(df_ticker):
    levels =      [0, 0.005, 0.01, 0.02, 0.03, 0.04, 0.05, 0.07]
    levels_tmos = [0, 0.005, 0.01, 0.015, 0.02, 0.03, 0.04, 0.05]

    #w1
    df_close_w1 = calc_stats_diff_1(df_ticker['close_1day'], feat_name='close_1day_w1')
    df_volume_w1 = calc_stats_diff_1(df_ticker['volume_1day'], feat_name='volume_1day_w1')
    df_tmos_close_w1 = calc_stats_diff_1(df_ticker['tmos_close_1day'], feat_name='tmos_close_1day_w1')
    assert df_ticker.shape[0] == df_close_w1.shape[0] == df_volume_w1.shape[0] == df_tmos_close_w1.shape[0], 'Error w1'
    
    #w3
    df_close_w3 = calc_stats(df_ticker['close_1day'], window=3, feat_name='close_1day_w3')
    df_volume_w3 = calc_stats(df_ticker['volume_1day'], window=3, feat_name='volume_1day_w3')
    df_tmos_close_w3 = calc_stats(df_ticker['tmos_close_1day'], window=3, feat_name='tmos_close_1day_w3')
    assert df_ticker.shape[0] == df_close_w3.shape[0] == df_volume_w3.shape[0] == df_tmos_close_w3.shape[0], 'Error w3'
    
    #w5
    df_close_w5 = calc_stats(df_ticker['close_1day'], window=5, feat_name='close_1day_w5')
    df_volume_w5 = calc_stats(df_ticker['volume_1day'], window=5, feat_name='volume_1day_w5')
    df_tmos_close_w5 = calc_stats(df_ticker['tmos_close_1day'], window=5, feat_name='tmos_close_1day_w5')
    assert df_ticker.shape[0] == df_close_w5.shape[0] == df_volume_w5.shape[0] == df_tmos_close_w5.shape[0], 'Error w5'
    
    #w20
    df_close_w20 = calc_stats(df_ticker['close_1day'], window=5*4, feat_name='close_1day_w20')
    df_close_levels_w20 = calc_levels(df_ticker['close_1day'], window=5*4, levels=levels, feat_name='close_1day_w20')
    df_volume_w20 = calc_stats(df_ticker['volume_1day'], window=5*4, feat_name='volume_1day_w20')
    df_tmos_close_w20 = calc_stats(df_ticker['tmos_close_1day'], window=5*4, feat_name='tmos_close_1day_w20')
    df_tmos_close_levels_w20 = calc_levels(df_ticker['tmos_close_1day'], window=5*4, levels=levels_tmos, feat_name='tmos_close_1day_w20')
    assert df_ticker.shape[0] == df_close_w20.shape[0] == df_close_levels_w20.shape[0] == df_volume_w20.shape[0] == df_tmos_close_w20.shape[0] == df_tmos_close_levels_w20.shape[0], 'Error w20'
    
    
    #w100
    df_close_w100 = calc_stats(df_ticker['close_1day'], window=100, feat_name='close_1day_w100')
    df_close_levels_w100 = calc_levels(df_ticker['close_1day'], window=100, levels=levels, feat_name='close_1day_w100')
    df_volume_w100 = calc_stats(df_ticker['volume_1day'], window=100, feat_name='volume_1day_w100')
    df_tmos_close_w100 = calc_stats(df_ticker['tmos_close_1day'], window=100, feat_name='tmos_close_1day_w100')
    df_tmos_close_levels_w100 = calc_levels(df_ticker['tmos_close_1day'], window=100, levels=levels_tmos, feat_name='tmos_close_1day_w100')
    assert df_ticker.shape[0] == df_close_w100.shape[0] == df_close_levels_w100.shape[0] == df_volume_w100.shape[0] == df_tmos_close_w100.shape[0] == df_tmos_close_levels_w100.shape[0], 'Error w100'
    
    df = pd.concat([df_ticker,
                       df_close_w1, df_volume_w1, df_tmos_close_w1, 
                       df_close_w3, df_volume_w3, df_tmos_close_w3, 
                       df_close_w5, df_volume_w5, df_tmos_close_w5,
                       df_close_w20, df_close_levels_w20, df_volume_w20, df_tmos_close_w20, df_tmos_close_levels_w20,
                       df_close_w100, df_close_levels_w100, df_volume_w100, df_tmos_close_w100, df_tmos_close_levels_w100], axis=1)
    
    assert (df_ticker.shape[0] == df.shape[0]) and (df.shape[1] == (df_ticker.shape[1]+3*df_close_w1.shape[1]+12*df_close_w5.shape[1]+4*df_close_levels_w20.shape[1])), 'Error concat'

    return df





def calculate_features_1hour(df_ticker):
    
    levels =      [0, 0.005, 0.01, 0.02, 0.03, 0.04, 0.05, 0.07]
    levels_tmos = [0, 0.005, 0.01, 0.015, 0.02, 0.03, 0.04, 0.05]

    #w1
    df_close_w1 = calc_stats_diff_1(df_ticker['close'], feat_name='close_w1')
    df_volume_w1 = calc_stats_diff_1(df_ticker['volume'], feat_name='volume_w1')
    df_tmos_close_w1 = calc_stats_diff_1(df_ticker['tmos_close'], feat_name='tmos_close_w1')
    assert df_ticker.shape[0] == df_close_w1.shape[0] == df_volume_w1.shape[0] == df_tmos_close_w1.shape[0], 'Error w1'
    
    #w5
    df_close_w5 = calc_stats(df_ticker['close'], window=5, feat_name='close_w5')
    df_volume_w5 = calc_stats(df_ticker['volume'], window=5, feat_name='volume_w5')
    df_tmos_close_w5 = calc_stats(df_ticker['tmos_close'], window=5, feat_name='tmos_close_w5')
    assert df_ticker.shape[0] == df_close_w5.shape[0] == df_volume_w5.shape[0] == df_tmos_close_w5.shape[0], 'Error w5'
    
    #w14
    df_close_w14 = calc_stats(df_ticker['close'], window=14, feat_name='close_w14')
    df_volume_w14 = calc_stats(df_ticker['volume'], window=14, feat_name='volume_w14')
    df_tmos_close_w14 = calc_stats(df_ticker['tmos_close'], window=14, feat_name='tmos_close_w14')
    assert df_ticker.shape[0] == df_close_w14.shape[0] == df_volume_w14.shape[0] == df_tmos_close_w14.shape[0], 'Error w14'

    #w70=14*5
    df_close_w70 = calc_stats(df_ticker['close'], window=70, feat_name='close_w70')
    df_close_levels_w70 = calc_levels(df_ticker['close'], window=70, levels=levels, feat_name="close_w70")
    df_volume_w70 = calc_stats(df_ticker['volume'], window=70, feat_name='volume_w70')
    df_tmos_close_w70 = calc_stats(df_ticker['tmos_close'], window=70, feat_name='tmos_close_w70')
    df_tmos_close_levels_w70 = calc_levels(df_ticker['tmos_close'], window=70, levels=levels_tmos, feat_name='tmos_close_w70')
    assert df_ticker.shape[0] == df_close_w70.shape[0] == df_close_levels_w70.shape[0] == df_volume_w70.shape[0] == df_tmos_close_w70.shape[0] == df_tmos_close_levels_w70.shape[0], 'Error w70'


    
    df = pd.concat([df_ticker,
                       df_close_w1, df_volume_w1, df_tmos_close_w1,
                       df_close_w5, df_volume_w5, df_tmos_close_w5,
                       df_close_w14, df_volume_w14, df_tmos_close_w14,
                       df_close_w70, df_close_levels_w70, df_volume_w70, df_tmos_close_w70, df_tmos_close_levels_w70
                      ], axis=1)
    
    assert (df_ticker.shape[0] == df.shape[0]) and (df.shape[1] == (df_ticker.shape[1]+3*df_close_w1.shape[1]+9*df_close_w5.shape[1]+2*df_close_levels_w70.shape[1])), 'Error concat'

    return df

In [34]:
def uniq_pairs(cols):
    pairs = []
    for i in range(len(cols)-1):
        for j in range(i+1, len(cols)):
            pairs += [(cols[i], cols[j])]
    return pairs

def calc_relative_features(df, groups):
    for group in groups:
        if type(group) == list:
            pairs = uniq_pairs(group)
            for pair in pairs:
                new_col = f'{pair[0]}/{pair[1]}'
                df[new_col] = df[pair[0]] / (df[pair[1]] + np.finfo(np.float32).eps)

        if type(group) == dict:
            pair1 = list(group.keys())[0]
            for pair0 in group[pair1]:
                new_col = f'{pair0}/{pair1}'
                df[new_col] = df[pair0] / (df[pair1] + np.finfo(np.float32).eps)

    return df

    




In [35]:
def prepare_data_lgbm(stocks, features):
    #logs
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'Start prepare data to lgbm' +'\n')

    day_name = str(pd.Timestamp.now().date()) 
    hour_name = str(pd.Timestamp.now()).split(':')[0].replace(' ', '_')

    #prepare
    data_lgbm = []

    #тут можно ускорить на минуту, если в одном общем датафрейме считать фичи (часовый и дневные)
    for ticker in tqdm(stocks):
        try:
            df_1day = load_pkl(f"./data/preproc/1day/{day_name}/{ticker}.pkl")
            df_1hour = load_pkl(f"./data/preproc/1hour/{hour_name}/{ticker}.pkl")

            df_1day = df_1day.rename(columns={col : col+'_1day' for col in df_1day.columns})            

            # вот тут
            df_1day = calculate_features_1day(df_1day).iloc[-1:].reset_index(drop=True)
            df_1hour = calculate_features_1hour(df_1hour).iloc[-1:].reset_index(drop=True)

            #time features
            df_1hour['hour'] = df_1hour['time'].dt.hour
            df_1hour['day'] = df_1hour['time'].dt.day
            df_1hour['weekday'] = np.minimum(df_1hour['time'].dt.dayofweek, 4) / 4

            assert not df_1day.isnull().any().any(), 'Error nulls df_1day'
            assert not df_1hour.isnull().any().any(), 'Error nulls df_1hour'
            
            data = pd.concat([df_1hour, df_1day], axis=1)
        
            data_lgbm += [data.copy()]
            
        except Exception as exception:
            write_logs(str(pd.Timestamp.now().round('s')) + ' '+ f'{ticker}: failed to prepare data to lgbm ({exception})' +'\n')

    try:
        data_lgbm = pd.concat(data_lgbm)
        data_lgbm = calc_relative_features(data_lgbm, GROUPS_1DAY+GROUPS_1HOUR)
        
        assert not data_lgbm.isnull().any().any(), 'Error nulls data_lgbm'  
        
        data_lgbm = data_lgbm[['time', 'ticker', 'close', 'time_1day'] + features].reset_index(drop=True).copy()
        dump_pkl(data_lgbm, f"./data/lgbm/data_lgbm_{hour_name}.pkl")

    except Exception as exception:
        write_logs(str(pd.Timestamp.now().round('s')) + ' '+ f'Failed to prepare data to lgbm ({exception})' +'\n')

    #logs
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'End prepare data to lgbm' +'\n')

In [36]:
#prepare_data_lgbm(STOCKS, FEATURES)

In [37]:
#df = load_pkl('./data/lgbm/data_lgbm_2025-02-04_16.pkl')

# 6. predict lgbm

In [39]:
class lgbm_model:
    def __init__(self, name=None, strategy=None, threshold=None, path=None, features=None):
        self.name = name
        self.strategy = strategy
        self.threshold = threshold
        self.model = load_pkl(path)
        self.features = features
        
    def predict(self, data_cp):
        data = data_cp.copy()

        data['model'] = self.name
        
        data['y_pred'] = self.model.predict(data[self.features])
        data['y_pred_bin'] = (data['y_pred'].values >= self.threshold).astype(int)
    
        #можно для больших чисел без 6 знаков после запятой сделать
        data['fix_lose'] = np.round(data['close'] * self.strategy[0], 6)
        data['fix_win'] = np.round(data['close'] * self.strategy[1], 6)
    
        data = data[['time', 'ticker', 'close', 'model', 'y_pred', 'y_pred_bin', 'fix_lose', 'fix_win']]

        return data
        

In [40]:
MODEL1 = lgbm_model(name='long_-2to+4_2days', 
                    strategy=(0.98, 1.04), 
                    threshold=0.3668169577019395, #99
                    path='../bst_mdl/long_-2to+4_2days/model.pkl', 
                    features=MODEL_FEATURES1)


MODEL2 = lgbm_model(name='short_+0.5to-1_1day', 
                    strategy=(1.005, 0.99), 
                    threshold=0.5695387939626755, #95
                    path='../bst_mdl/short_+0.5to-1_1day/model.pkl', 
                    features=MODEL_FEATURES2)

In [41]:
def predict_lgbm(stocks):
    #logs
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'Start lgbm predicting' +'\n')

    try:
        hour_name = str(pd.Timestamp.now()).split(':')[0].replace(' ', '_')
        data = load_pkl(f"./data/lgbm/data_lgbm_{hour_name}.pkl")

        models = [MODEL1, MODEL2]

        data_result = []
        for model in models:
            data_result += [model.predict(data)]
            
        data_result = pd.concat(data_result).reset_index(drop=True)
        
        dump_pkl(data_result, f"./data/result/data_result_{hour_name}.pkl")
    except Exception as exception:
        write_logs(str(pd.Timestamp.now().round('s')) + ' ' + f'Failed lgbm predicting ({exception})' +'\n')

    #logs
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'End lgbm predicting' +'\n')

In [42]:
# predict_lgbm(STOCKS)

In [43]:
# df = load_pkl('./data/result/data_result_2025-02-04_16.pkl')
# df

# 7. Notification in telegram

In [45]:
import re
def escape_markdown(text):
    """функция для экранирования символов перед отправкой в маркдауне телеграма"""
    pattern = r"([_*\[\]()~|`])"
    return re.sub(pattern, r"\\\1", text)

def telegram_bot_sendtext(bot_message):
    bot_message = escape_markdown(bot_message)
    send_text = 'https://api.telegram.org/bot' + BOT_TOKEN + '/sendMessage?chat_id=' + BOT_CHAT_ID + '&parse_mode=Markdown&text=' + bot_message
    response = requests.get(send_text)
    return response.json()


# test = telegram_bot_sendtext("Testing Telegram bot")
# print(test)

In [46]:
#load_pkl('./data/result/data_result_2025-02-04_16.pkl')

In [47]:
def send_notification_tg(stocks):    
    #logs
    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'Start sending msg' +'\n')
    
    try:
        hour_name = str(pd.Timestamp.now()).split(':')[0].replace(' ', '_')
        data = load_pkl(f"./data/result/data_result_{hour_name}.pkl")

        msg = f'signals: {data['y_pred_bin'].sum()}\n'

        for model in data['model'].unique():
            msg += f"\n{model}\n\n"
            data_model = data[data['model'] == model]
            for i, row in data_model.iterrows():
                if row['y_pred_bin'] == 1:
                    msg += f"{row['ticker']}, close = {row['close']}\n"
                    sign = '<' if row['fix_lose'] < row['fix_win'] else '>'
                    msg += f"{row['fix_lose']} {sign} {row['fix_win']}\n\n"

        msg += '\nno data:\n'
        no_data_stocks = set(stocks).difference(set(data['ticker'].tolist()))
        msg += "\n".join(list(no_data_stocks))
        
        _ = telegram_bot_sendtext(msg)
              
    except Exception as exception:
        write_logs(str(pd.Timestamp.now().round('s')) + ' ' + f'Failed sending msg ({exception})' +'\n')

    write_logs(str(pd.Timestamp.now().round('s')) + ' ' + 'End sending msg' +'\n')
    

In [48]:
# send_notification_tg(STOCKS)

# x. Daemon Khibiny

In [50]:
def is_business_day(date):
    return bool(len(pd.bdate_range(date, date)))

def flag_work_1day(date):
    time_name = str(pd.Timestamp.now().date()) 
    if (date.hour >= 10) and (date.minute >= 1) and is_business_day(date):
        if (not os.path.exists(f'./data/raw/1day/{time_name}')):# if deamon is starting in the middle of the day
            return True
        if len(os.listdir(f'./data/raw/1day/{time_name}')) == 0: #если произошла ошибка при первой попытке скачивания данных
            return True

    if (date.hour == 10) and (date.minute == 50) and is_business_day(date): #дополнительное скачивнаие данных за пред дни для низколиквидных акций
        return True
        
    return False
    
def flag_work_1hour(date):
    if date.hour == 10 and date.minute == 3 and is_business_day(date): #отдельный случай, чтобы успели скачаться дневные данные (для проверки актуальности данных)
        return True
    if date.hour in range(11, 24) and date.minute == 1 and is_business_day(date):
        return True
    return False   


In [ ]:
while 1 > 0:
    clear_output()
    cur_time = pd.Timestamp.now().floor('s')
    print(cur_time)
    
    if flag_work_1day(cur_time):
        get_data_1day(STOCKS)
        preproc_data_1day(STOCKS)
        
    if flag_work_1hour(cur_time):   
        get_data_1hour(STOCKS)
        preproc_data_1hour(STOCKS)
        prepare_data_lgbm(STOCKS, FEATURES)
        predict_lgbm(STOCKS)
        send_notification_tg(STOCKS)


    time.sleep(10)

2025-02-14 12:26:11


In [ ]:
        # get_data_1hour(STOCKS)
        # preproc_data_1hour(STOCKS)
        # prepare_data_lgbm(STOCKS, FEATURES)
        # predict_lgbm(STOCKS)
        # send_notification_tg(STOCKS)

In [ ]:
#get_data_1hour(STOCKS)

In [ ]:
#preproc_data_1hour(STOCKS)

In [ ]:
#get_data_1hour(STOCKS)

In [ ]:
data = load_pkl('./data/result/data_result_2025-02-10_15.pkl')
data

In [ ]:
data['y_pred'].max()